Reference: https://www.geeksforgeeks.org/nlp/amazon-product-reviews-sentiment-analysis-in-python/


In [4]:
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import nltk
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

from datetime import datetime



# Download required NLTK data (run once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize once for all processes
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1035)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1035)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1035)>


In [5]:
categories = [
    "All_Beauty",
    "Amazon_Fashion",
    "Appliances",
    "Arts_Crafts_and_Sewing",
    "Automotive",
    "Baby_Products",
    "Beauty_and_Personal_Care",
    "Books",
    "CDs_and_Vinyl",
    "Cell_Phones_and_Accessories",
    "Clothing_Shoes_and_Jewelry",
    "Digital_Music",
    "Electronics",
    "Gift_Cards",
    "Grocery_and_Gourmet_Food",
    "Handmade_Products",
    "Health_and_Household",
    "Health_and_Personal_Care",
    "Home_and_Kitchen",
    "Industrial_and_Scientific",
    "Kindle_Store",
    "Magazine_Subscriptions",
    "Movies_and_TV",
    "Musical_Instruments",
    "Office_Products",
    "Patio_Lawn_and_Garden",
    "Pet_Supplies",
    "Software",
    "Sports_and_Outdoors",
    "Subscription_Boxes",
    "Tools_and_Home_Improvement",
    "Toys_and_Games",
    "Video_Games",
    "Unknown"
]

In [6]:
def clean_text(text):
    """
    Clean and preprocess text data
    - Convert to lowercase
    - Tokenize into words
    - Remove stopwords, punctuation, non-alphabetic tokens
    - Lemmatize words
    """
    text = text.lower().strip()
    tokens = word_tokenize(text)
    cleaned_tokens = [
        lemmatizer.lemmatize(w)
        for w in tokens
        if w not in stop_words and w not in string.punctuation and w.isalpha()
    ]
    return cleaned_tokens

In [7]:
# load data of all categories; store in the form that includes: category, reviews and sentiment level (1-5)

# we load 5000 reviews from each category, then sample 100 reviews for each rating (1-5), total 500 reviews per category
def process_category(category, sample_size_per_rating=100, reservoir_size=5000):
    """ return a dictionary with the structue: {"category":[
      {"category":[name],"tokens":[......], "rating":[1]},{...}
      ]}"""
    try:

      # load the data from hugging face
        data_url = f"https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/review_categories/{category}.jsonl"

        dataset = load_dataset("json", data_files=data_url, split="train", streaming=True)

        raw_rows = []
        for i, row in enumerate(dataset):
            if i >= reservoir_size:
                break

            raw_rows.append({
                'text': row.get('text', ''),
                'rating': row.get('rating', row.get('overall', None)),
                'title': row.get('title', ''),
                'images': row.get('images', []),
                'verified_purchase': row.get('verified_purchase', False),
                'asin': row.get('asin', ''),
                'parent_asin': row.get('parent_asin', ''),
                'user_id': row.get('user_id', ''),
                'timestamp': row.get('timestamp', None),
                'helpful_vote': row.get('helpful_vote', 0)
            })

        # set a seed for randomization
        random.seed(42)
        
        # Group raw_rows by rating (1-5)
        rating_groups = {}
        for row in raw_rows:
            rating = row['rating']
            if rating not in rating_groups:
                rating_groups[rating] = []
            rating_groups[rating].append(row)
        
        # Sample sample_size_per_rating (100) from each rating (1-5)
        sampled_rows = []
        for rating in range(1, 6):
            if rating in rating_groups:
                sample = random.sample(rating_groups[rating], min(sample_size_per_rating, len(rating_groups[rating])))
                sampled_rows.extend(sample)

        # clean the data: tokenize each review
        processed_data = []
        for row in sampled_rows:
            tokens = clean_text(row['text'])
            title = row['title'].lower().strip()
            time = datetime.fromtimestamp(row['timestamp']/ 1000)

            if tokens:
                processed_data.append({
                    'category': category,
                    'tokens': tokens,
                    'rating': row['rating'],
                    'original_text': row['text'],
                    'title': title,
                    'images': row['images'],
                    'verified_purchase': row['verified_purchase'],
                    'asin': row['asin'],
                    'parent_asin': row['parent_asin'],
                    'user_id': row['user_id'],
                    'datetime': time,
                    'helpful_vote': row['helpful_vote']
                })

        print(f"{category}: {len(processed_data)} reviews processed")
        return category, processed_data

    except Exception as e:
        print(f"{category}: Error - {str(e)}")
        return category, []
#

In [8]:
nltk.download('punkt_tab')

[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1035)>


False

In [9]:
# execute
category_collection = {}

with ThreadPoolExecutor(max_workers=4) as executor:
    futures = {executor.submit(process_category, cat): cat for cat in categories}
    for future in as_completed(futures):
        cat, data_list = future.result()
        category_collection[cat] = data_list

All_Beauty: Error - 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/Users/yifeicao/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.13/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.13/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.13/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Amazon_Fashion: Error - 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>>

All_Beauty: Error - 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/Users/yifeicao/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.13/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.13/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.13/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Amazon_Fashion: Error - 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>>

KeyboardInterrupt: 

In [ ]:
final_list = []
for cat, items in category_collection.items():
    for item in items:
        final_list.append({
            'category': item['category'],
            'rating': item['rating'],
            'tokens': item['tokens'],
            'text_cleaned': ' '.join(item['tokens']),
            'original_text': item['original_text'],
            'token_count': len(item['tokens']),
            'title': item['title'],
            'images': item['images'],
            'verified_purchase': item['verified_purchase'],
            'asin': item['asin'],
            'parent_asin': item['parent_asin'],
            'user_id': item['user_id'],
            'datetime': item['datetime'],
            'helpful_vote': item['helpful_vote']
        })

df = pd.DataFrame(final_list)

In [ ]:
df.head()

,category,rating,sentiment,tokens,text_cleaned,original_text,token_count,title,images,verified_purchase,asin,parent_asin,user_id,datetime,helpful_vote
0,Appliances,5.0,positive,"[exelent, save, coffee, water, waste, easy, cl...",exelent save coffee water waste easy clean act...,"Exelent, save on coffee, water, no waste, e...",14,savings,[],True,B00LGEKOMS,B07RNJY499,AEOVCZC77QZJQPBIAIKCFV7AS7PA,2017-10-16 22:46:40.529,0
1,Appliances,5.0,positive,"[ordered, wrong, part, quality, part, seemed, ...",ordered wrong part quality part seemed good,Ordered wrong part but the quality of this par...,7,check part numbers,[],True,B094YWPF68,B094YWPF68,AEU2V36H3G45EFVLASUPD56B7ATQ,2021-08-30 19:50:53.564,0
2,Appliances,5.0,positive,[described],described,as described,1,five stars,[],True,B00LQL043A,B00LQL043A,AE7FJMYY4AKWBDASLTMMQ5WASB7A,2016-09-28 15:08:18.000,0
3,Appliances,5.0,positive,"[used, replace, broken, door, bin, back, ice, ...",used replace broken door bin back ice dispense...,Used to replace a broken door bin on the back ...,18,fits,[],True,B00C29G3N0,B00C29G3N0,AGM4WN3EOAA3RUAUXGH2S2AUL6WA,2019-01-06 21:01:13.614,3
4,Appliances,5.0,positive,"[husband, us, broke, cleaning, replace, well, ...",husband us broke cleaning replace well say one...,My husband uses these. I broke it when cleanin...,35,easy peasy to use and clean.,[],True,B01DP1IWKU,B092LLM7H3,AEBXJRP4COCKP22LPKUDVCQ7JKVQ,2019-08-09 02:13:50.975,0


In [ ]:
df.to_parquet('amazon_user_reviews_100/reivew.parquet', index=False)